# Part4: 全连接神经网络

In [1]:
import tensorflow as tf
import pickle
import numpy as np
import os

# the path of the cifar10 datasets 
# which is downloaded from the link of "https://www.cs.toronto.edu/~kriz/cifar.html" (Warning: we should download the python version)
CIFAR_DIR = "./cifar-10-batches-py" 
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [11]:
def load_data(filename):
    # read data from data file
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

class CifarData(object):
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data) # 纵向合并 [10000, 3072]，每个像素点的大小是（0，255）
        # 将数据缩放到 （-1，1）之间
        # 255 / 127.5 - 1 = 1
        # 0 / 127.5 -1 = -1
        self._data = self._data / 127.5 -1 
        print(self._data.shape)
        self._labels = np.hstack(all_labels) # 横向合并 [10000,]
        self._num_examples = self._data.shape[0] # 10000
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # 打乱顺序，用于 training set
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        '''return batch_size examples as a batch'''
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                # 训练集的情况，要 shuffle 一下
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                # 测试集的情况，不需要 shuffle
                self._indicator = 0
                end_indicator = batch_size

        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_lables = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_lables

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(10000, 3072)


# 网络结构：三层 全连接（dense）

In [12]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

hidden1 = tf.layers.dense(x, 100, activation=tf.nn.relu)       # [None, 3072] * [3072, 100] = [None, 100]
hidden2 = tf.layers.dense(hidden1, 100, activation=tf.nn.relu) # [None, 100] * [100 * 100] = [None, 100]
hidden3 = tf.layers.dense(hidden2, 50, activation=tf.nn.relu)  # [None, 100] * [100 * 50] = [None, 50]

# 最后输出 10 个，因为有 10 个分类
y_ = tf.layers.dense(hidden3, 10) # [None, 50] * [50 * 10] = [None, 10]

# 采用 sofmax 函数进行多分类
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [13]:
init = tf.global_variables_initializer()
# hyper-parameter setting
batch_size = 20
train_steps = 100000 # 训练集个数50000 / 20 （batch_size） = 2500 step 训练完全部训练集，100000 / 2500 = 40 次，每次都打乱
test_steps = 500     # 测试集个数10000 / 20（batch_size） = 500 step

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_lables = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_lables}
        )
        if (i + 1) % 500 == 0:
            print("[Trian] Step: %d, loss: %4.5f, acc: %4.5f" % (i+1, loss_val, acc_val))
        if (i + 1) % 5000 == 0:
            all_test_acc_val = [] # 存储每个 batch 的准确率
            for j in range(test_steps):
                test_batch_data, test_batch_lables = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_lables
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i + 1, test_acc))

[Trian] Step: 500, loss: 1.89492, acc: 0.20000
[Trian] Step: 1000, loss: 1.78843, acc: 0.30000
[Trian] Step: 1500, loss: 1.94426, acc: 0.45000
[Trian] Step: 2000, loss: 1.66395, acc: 0.35000
[Trian] Step: 2500, loss: 2.14697, acc: 0.25000
[Trian] Step: 3000, loss: 1.53480, acc: 0.50000
[Trian] Step: 3500, loss: 1.25918, acc: 0.60000
[Trian] Step: 4000, loss: 1.19025, acc: 0.55000
[Trian] Step: 4500, loss: 1.13777, acc: 0.65000
[Trian] Step: 5000, loss: 1.25756, acc: 0.60000
[Test] Step: 5000, acc: 0.48100
[Trian] Step: 5500, loss: 1.33123, acc: 0.55000
[Trian] Step: 6000, loss: 1.09452, acc: 0.60000
[Trian] Step: 6500, loss: 1.49408, acc: 0.45000
[Trian] Step: 7000, loss: 1.35942, acc: 0.40000
[Trian] Step: 7500, loss: 1.30791, acc: 0.45000
[Trian] Step: 8000, loss: 1.38943, acc: 0.60000
[Trian] Step: 8500, loss: 1.07217, acc: 0.55000
[Trian] Step: 9000, loss: 1.44326, acc: 0.45000
[Trian] Step: 9500, loss: 1.28944, acc: 0.65000
[Trian] Step: 10000, loss: 1.16935, acc: 0.65000
[Test] S

[Trian] Step: 79500, loss: 0.63747, acc: 0.80000
[Trian] Step: 80000, loss: 0.77208, acc: 0.70000
[Test] Step: 80000, acc: 0.50210
[Trian] Step: 80500, loss: 0.30086, acc: 0.95000
[Trian] Step: 81000, loss: 0.75258, acc: 0.80000
[Trian] Step: 81500, loss: 0.86291, acc: 0.70000
[Trian] Step: 82000, loss: 0.52131, acc: 0.85000
[Trian] Step: 82500, loss: 0.78882, acc: 0.75000
[Trian] Step: 83000, loss: 0.28343, acc: 0.95000
[Trian] Step: 83500, loss: 0.53810, acc: 0.80000
[Trian] Step: 84000, loss: 0.51719, acc: 0.85000
[Trian] Step: 84500, loss: 0.96530, acc: 0.60000
[Trian] Step: 85000, loss: 0.52802, acc: 0.85000
[Test] Step: 85000, acc: 0.50530
[Trian] Step: 85500, loss: 0.60971, acc: 0.75000
[Trian] Step: 86000, loss: 0.62556, acc: 0.80000
[Trian] Step: 86500, loss: 0.61695, acc: 0.85000
[Trian] Step: 87000, loss: 0.37981, acc: 0.85000
[Trian] Step: 87500, loss: 0.41836, acc: 0.90000
[Trian] Step: 88000, loss: 0.96320, acc: 0.65000
[Trian] Step: 88500, loss: 0.81386, acc: 0.80000
[Tr